In [1]:
# images
import os
import numpy as np
import scipy.io
from datasets import load_dataset, Dataset, Image

image_folder = './data/jpg/'
image_files = [os.path.join(image_folder, file) for file in os.listdir(image_folder)]

labels_file = "./data/imagelabels.mat"
labels_data = scipy.io.loadmat(labels_file)

labels = labels_data['labels'][0]
labels = labels - 1 # Adjusting labels to be in the range 0-101

# Create a dictionary first
data_dict = {"image": image_files, "label": labels.tolist()}
# Now create the Dataset
dataset = Dataset.from_dict(data_dict)

# Cast the image column to Image format
dataset = dataset.cast_column("image", Image())
print(type(dataset))
print(dataset[0])
print(len(dataset))
entry = dataset["image"][0]
print(entry)  # {'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=512x512 at 0x7F52AFC8AC50>, 'label': 79}

<class 'datasets.arrow_dataset.Dataset'>
{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=721x500 at 0x7FB92271EF50>, 'label': 76}
8189


In [ ]:
print(type(dataset))
print(len(dataset))


In [ ]:
dataset = dataset.train_test_split(test_size=0.2)
print(len(dataset))
print(type(dataset))
print(dataset.keys())
print(len(dataset['train']))
print(len(dataset['test']))

In [ ]:
import torch

# some code that uses GPU memory
# ...

# After finishing with a batch of data or an epoch, clear the cache
torch.cuda.empty_cache()


In [ ]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()


from transformers import AutoImageProcessor

checkpoint = "google/vit-base-patch16-224-in21k"
image_processor = AutoImageProcessor.from_pretrained(checkpoint)

from torchvision.transforms import RandomResizedCrop, Compose, Normalize, ToTensor

normalize = Normalize(mean=image_processor.image_mean, std=image_processor.image_std)
size = (
    image_processor.size["shortest_edge"]
    if "shortest_edge" in image_processor.size
    else (image_processor.size["height"], image_processor.size["width"])
)
_transforms = Compose([RandomResizedCrop(size), ToTensor(), normalize])

def transforms(examples):
    examples["pixel_values"] = [_transforms(img.convert("RGB")) for img in examples["image"]]
    del examples["image"]
    return examples

dataset= dataset.with_transform(transforms)

import evaluate

accuracy = evaluate.load("accuracy")

import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

from transformers import AutoModelForImageClassification, TrainingArguments, Trainer

model = AutoModelForImageClassification.from_pretrained(
    checkpoint,
    num_labels=len(labels)#,
   # id2label=id2label,
    #label2id=label2id,
)

training_args = TrainingArguments(
    output_dir="vit_model",
    remove_unused_columns=False,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=8, #16
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=8, #16
    num_train_epochs=3,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=image_processor,
    compute_metrics=compute_metrics,
)

trainer.train()

